# ** RFM Customer Segmentation & Cohort Analysis Project **  

## Determines

Using the [Online Retail dataset](https://archive.ics.uci.edu/ml/datasets/Online+Retail) from the UCI Machine Learning Repository for exploratory data analysis, ***Customer Segmentation***, ***RFM Analysis***, ***K-Means Clustering*** and ***Cohort Analysis***.

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

Feature Information:

**InvoiceNo**: Invoice number. *Nominal*, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. 
<br>
**StockCode**: Product (item) code. *Nominal*, a 5-digit integral number uniquely assigned to each distinct product.
<br>
**Description**: Product (item) name. *Nominal*. 
<br>
**Quantity**: The quantities of each product (item) per transaction. *Numeric*.
<br>
**InvoiceDate**: Invoice Date and time. *Numeric*, the day and time when each transaction was generated.
<br>
**UnitPrice**: Unit price. *Numeric*, Product price per unit in sterling.
<br>
**CustomerID**: Customer number. *Nominal*, a 5-digit integral number uniquely assigned to each customer.
<br>
**Country**: Country name. *Nominal*, the name of the country where each customer resides.


# 1. Data Cleaning & Exploratory Data Analysis

## Import Modules, Load Data & Data Review

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Customer Segmentation/Online Retail.xlsx")

In [ ]:
df.shape # Shape of data

In [ ]:
df.head(5) 

In [ ]:
df.info() 

In [ ]:
df.isnull().sum()/ len(df) * 100 

Description and CustomerID columns have missing values.

In [ ]:
df.describe() 

Quantity and UnitPrice columns have negative values. 

### i. Take a look at relationships between InvoiceNo, Quantity and UnitPrice columns.

In [ ]:
df["InvoiceNo"].value_counts(dropna=False)

In [ ]:
df["Quantity"].value_counts(dropna=False)

In [ ]:
df["UnitPrice"].value_counts(dropna=False)

We see that there are negative values in the Quantity and UnitPrice columns. These are possibly canceled and returned orders. Let's check it out.

In [ ]:
df_three = df[["InvoiceNo", "Quantity", "UnitPrice"]] # Created a new dataframe for InvoiceNo, Quantity, UnitPrice columns
df_three[df_three["Quantity"] < 0] # Quantity lower than 0

In [ ]:
df_three[(df_three["Quantity"] < 0) & (df_three["InvoiceNo"].str.contains("C"))] # InvoiceNo startswith C and Quantity lower than 0

In [ ]:
df_three[(df_three["Quantity"] < 0) & (df_three["InvoiceNo"].str.contains("C") == False)]

There are 9192 rows that Quantity is lower than 0 and 7856 rows's InvoiceNo starts with "C"

In [ ]:
df_three[df_three["UnitPrice"] < 0] # UnitPrice lower than 0

In [ ]:
df_three[df_three["InvoiceNo"].str.contains("A") == True] # InvoiceNo contains A

In [ ]:
df_three[(df_three["UnitPrice"] < 0) & (df_three["InvoiceNo"].str.contains("A"))] # UnitPrice is lower than 0 and InvoiceNo starts with A

In [ ]:
df_three[(df_three["UnitPrice"] > 0) & (df_three["InvoiceNo"].str.contains("A") == True)] # UnitPrice is greater than 0 and InvoiceNo starts with A

There are 2 rows that UnitPrice column is lower than 0  and  3 columns start with A.

In [ ]:
print("The number of rows that InvoiceNo starts with C:", df["InvoiceNo"].str.contains("C").sum())
print("The number of rows that InvoiceNo starts with A:", df["InvoiceNo"].str.contains("A").sum())
print("The number of rows that Quantity lower than 0:", sum(df["Quantity"] < 0))
print("The number of rows that UnitPrice lower than 0:", sum(df["UnitPrice"] < 0))

### ii. What does the letter "C" in the InvoiceNo column mean?




In [ ]:
df[df["InvoiceNo"].str.contains("C") == True]

In [ ]:
df[(df["Quantity"] < 0) & (df["InvoiceNo"].str.contains("C"))].shape[0]

In [ ]:
df[(df["InvoiceNo"].str.contains("C")) & (df["Quantity"] > 0)].shape[0]

If the invoice number starts with the letter "C", it means the order was cancelled. Or those who abandon their order. When we filter canceled orders by Quantity > 0 or filter non-canceled orders by Quantity < 0 nothing returns, this confirms that negative values mean the order was canceled. 

In [ ]:
df[df["UnitPrice"] < 0]

In [ ]:
df[df["InvoiceNo"].str.contains("A") == True]

In [ ]:
df[(df["UnitPrice"] < 0) & (df["InvoiceNo"].str.contains("A"))]

In [ ]:
df[df["StockCode"] == "B"]

Negative UnitPrice refers to doubtful process and their CustomerID's are missing values. Also StockCode's are B and InvoiceNo starts with A. This refers to doubtful process.


### iii. Handling Missing Values and Clean the Data from the Noise and Missing Values


In [ ]:
df.isnull().sum()

Since the customer ID's are missing, lets assume these orders were not made by the customers already in the data set because those customers already have ID's. 

We also don't want to assign these orders to those customers because this would alter the insights we draw from the data. 


In [ ]:
df2 = df # In the further researches, this can be helpful.

In [ ]:
df.shape

In [ ]:
df["Description"].fillna("", inplace=True) # Fill Description column with "" 
df.dropna(inplace=True) # Drop missing values from Dataset. This will drop rows which CustomerID is null.

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

### Exploring the Orders


1. The unique number of InvoiceNo  per customer

In [ ]:
print("The number of unique InvoiceNo per each customer:", df["InvoiceNo"].nunique() / df["CustomerID"].nunique())

2. What's the average number of unique items per order or per customer?

In [ ]:
print("Per order, average number of unique items:" ,df.groupby("InvoiceNo")["StockCode"].nunique().mean()) 

3. Let's see how this compares to the number of unique products per customer.

In [ ]:
print("Per customer, average number of unique items:" ,df.groupby("CustomerID")["StockCode"].nunique().mean()) 

### vi. Explore Customers by Country

1. What's the total revenue per country?

In [ ]:
plt.figure(figsize=(24,12))
plt.title("Total Revenue per Country", c="blue", size=14)
plt.xticks(rotation=45)
total_revenue_per_country = df.groupby("Country")["UnitPrice"].sum().sort_values(ascending=False)
sns.barplot(x=total_revenue_per_country.index, y=total_revenue_per_country.values) 

2. Visualize number of customer per country

In [ ]:
plt.figure(figsize=(24,12))
plt.title("Total Customer per Country", c="blue", size=14)
plt.xticks(rotation=45)
total_customer_per_country = df.groupby("Country")["CustomerID"].nunique().sort_values(ascending=False)
sns.barplot(x=total_customer_per_country.index, y=total_customer_per_country.values) 

3. Visualize total cost per country

In [ ]:
df["Cost"] = df["Quantity"] * df["UnitPrice"] # Created Cost column with multiplying Quantity and UnitPrice

In [ ]:
plt.figure(figsize=(24,12))
plt.title("Total Cost per Country", c="blue", size=14)
plt.xticks(rotation=45)
total_cost_per_country = df.groupby("Country")["Cost"].sum().sort_values(ascending=False)
sns.barplot(x=total_cost_per_country.index, y=total_cost_per_country.values) 

#### The UK not only has the most sales revenue, but also the most customers. Since the majority of this data set contains orders from the UK, we can explore the UK market further by finding out what products the customers buy together and any other buying behaviors to improve our sales and targeting strategy.

### vii. Explore the UK Market


1. Creating df_uk DataFrame

In [ ]:
df_uk = df[df["Country"] == "United Kingdom"]

2. What are the most popular products that are bought in the UK?

In [ ]:
df_uk[df_uk["StockCode"] == df_uk["StockCode"].value_counts().index[0]]["Description"].value_counts()

### We will continue analyzing the UK transactions with customer segmentation.

# 2. RFM Analysis

**[RFM](https://www.putler.com/rfm-analysis/) (Recency, Frequency, Monetary) Analysis** is a customer segmentation technique for analyzing customer value based on past buying behavior. RFM analysis was first used by the direct mail industry more than four decades ago, yet it is still an effective way to optimize your marketing.


- RECENCY (R): Time since last purchase
- FREQUENCY (F): Total number of purchases
- MONETARY VALUE (M): Total monetary value

Benefits of RFM Analysis

- Increased customer retention
- Increased response rate
- Increased conversion rate
- Increased revenue

RFM Analysis answers the following questions:
 - Who are our best customers?
 - Who has the potential to be converted into more profitable customers?
 - Which customers do we need to retain?
 - Which group of customers is most likely to respond to our marketing campaign?
 

In [ ]:
import datetime as dt
import re

### ii. Review df_uk DataFrame

In [ ]:
df_uk.head(5)

### iii. Recency: Days since last purchase

1. Choosing a date as a point of reference to evaluate how many days ago was the customer's last purchase.

In [ ]:
new_date = dt.datetime.strptime("2011-12-10", "%Y-%m-%d") # Choosing a date, I chose the 1 day later of last date in dataset.
new_date = new_date.date()

2. Creating a new column called Date which contains the invoice date without the timestamp

In [ ]:
df_uk["Date"] = df_uk["InvoiceDate"].dt.date 

3. Group by CustomerID and check the last date of purchase

In [ ]:
last_date_of_purchase = df_uk.groupby("CustomerID")["Date"].max()

In [ ]:
last_date_of_purchase.head()

4. Calculate the days since last purchase

In [ ]:
recency = new_date - last_date_of_purchase
recency = recency.astype("str")

def get_number(x):
  return re.findall("\d+", x)[0]

recency = recency.apply(get_number)
recency.head()

6. Plot RFM distributions

In [ ]:
plt.figure(figsize=(24,12))
plt.title("RFM Distributions for Recency", c="blue", size=14)
sns.histplot(recency.sort_values().values, color="Red")

### iv. Frequency: Number of purchases

1. Copy of df_uk and drop duplicates

In [ ]:
df_uk_copy = df_uk.drop_duplicates()
print("df_uk shape:",df_uk.shape)
print("df_uk_copy shape:", df_uk_copy.shape)

2. The frequency of purchases

In [ ]:
frequency = df_uk_copy.groupby("CustomerID")["InvoiceNo"].count()
frequency.head()

3. Plot RFM distributions

In [ ]:
plt.figure(figsize=(24,12))
plt.title("RFM Distributions for Frequency", c="blue", size=14)
sns.histplot(frequency.sort_values(ascending=False).values, color="Red")

### v. Monetary: Total amount of money spent

The monetary value is calculated by adding together the cost of the customers' purchases.


1. Calculate sum total cost by customers and named "Monetary"

In [ ]:
monetary = df_uk.groupby("CustomerID")["Cost"].sum()
monetary.head()

In [ ]:
df[df["CustomerID"] == 12346]

2. Plot RFM distributions

In [ ]:
plt.figure(figsize=(24,12))
plt.title("RFM Distributions for Monetary", c="blue", size=14)
sns.histplot(monetary.sort_values(ascending=False).values, color="Red")

### vi. Creating RFM Table

In [ ]:
df_rfm = pd.concat([recency, frequency, monetary], axis=1)
df_rfm = df_rfm.rename(columns={"Date":"Recency", "InvoiceNo":"Frequency", "Cost":"Monetary"})
df_rfm["Recency"] = df_rfm["Recency"].astype("int")
df_rfm.head()

# 3. Customer Segmentation with RFM Scores

##  RFM Score

The simplest way to create customer segments from an RFM model is by using **Quartiles**. We will assign a score from 1 to 4 to each category (Recency, Frequency, and Monetary) with 4 being the highest/best value. The final RFM score is calculated by combining all RFM values. For Customer Segmentation, you will use the df_rfm data set resulting from the RFM analysis.


1. Dividing the df_rfm into quarters

In [ ]:
df_rfm

### i. Creating the RFM Segmentation Table


1. Functions

In [ ]:
def recency_quarter(x):
  recency_quantile = df_rfm["Recency"].quantile([0.25, 0.50, 0.75]).values 
  if x <= recency_quantile[0]:
    return 4
  elif x <= recency_quantile[1]:
    return 3
  elif x <= recency_quantile[2]:
    return 2
  else:
    return 1

In [ ]:
def frequency_quarter(x):
  frequency_quantile = df_rfm["Frequency"].quantile([0.25, 0.50, 0.75]).values 
  if x <= frequency_quantile[0]:
    return 1
  elif x <= frequency_quantile[1]:
    return 2
  elif x <= frequency_quantile[2]:
    return 3
  else:
    return 4

In [ ]:
def monetary_quarter(x):
  monetary_quantile = df_rfm["Monetary"].quantile([0.25, 0.50, 0.75]).values 
  if x <= monetary_quantile[0]:
    return 1
  elif x <= monetary_quantile[1]:
    return 2
  elif x <= monetary_quantile[2]:
    return 3
  else:
    return 4

2. Scoring customers from 1 - 4 with functions

In [ ]:
df_rfm["RecencyScore"] = df_rfm["Recency"].apply(recency_quarter)
df_rfm["FrequencyScore"] = df_rfm["Frequency"].apply(frequency_quarter)
df_rfm["MonetaryScore"] = df_rfm["Monetary"].apply(monetary_quarter)

In [ ]:
df_rfm.head()

3. Combining scores for segmentation

In [ ]:
df_rfm["CombinedScore"] = df_rfm["RecencyScore"].astype("str") + df_rfm["FrequencyScore"].astype("str") + df_rfm["MonetaryScore"].astype("str")
df_rfm["CombinedScore"] = df_rfm["CombinedScore"].astype("int")
print("Combined Score unique values:", df_rfm["CombinedScore"].nunique())
df_rfm["CombinedScore"].head()

In [ ]:
df_rfm["CombinedScoreTotal"] = df_rfm["RecencyScore"] + df_rfm["FrequencyScore"] + df_rfm["MonetaryScore"]
df_rfm["CombinedScoreTotal"] = df_rfm["CombinedScoreTotal"].astype("int")
print("Combined Score Total unique values:", df_rfm["CombinedScoreTotal"].nunique())
df_rfm["CombinedScoreTotal"].head()

CombinedScore has 61 unique values. On the other hand, CombinedScoreTotal has 10 unique values. Labelling CombinedScoreTotal is more accountable.

In [ ]:
df_rfm.drop("CombinedScore", axis=1, inplace=True)

4. Defining rfm_level function that tags customers by using RFM_Scrores and Creating RFM Level

In [ ]:
df_rfm.head()

In [ ]:
def rfm_tags(x):
  if x <= 4:
    return "Requires Action"
  elif x <= 6:
    return "Needs Attention"
  elif x <= 8:
    return "Promising"
  elif x <= 10:
    return "Loyal"
  else:
    return "Best"

In [ ]:
df_rfm["rfm_level"] = df_rfm["CombinedScoreTotal"].apply(rfm_tags)
df_rfm["rfm_level"].head()

5. Value counts for RFM level and average combined score

In [ ]:
df_rfm.groupby("rfm_level")["CombinedScoreTotal"].count() # Size of each segment

In [ ]:
df_rfm.groupby("rfm_level")["CombinedScoreTotal"].mean() # Average Combined Score of each segment

## Plot RFM Segments

1. Creating Plot

In [ ]:
plt.figure(figsize=(12,8))
plt.title("Countplot of RFM Segments", c="blue", size=14)
sns.countplot(df_rfm["rfm_level"], palette="magma");

2. How many customers do we have in each segment?

In [ ]:
df_rfm.groupby("rfm_level")["CombinedScoreTotal"].count() # Size of each segment

# 3. Applying K-Means Clustering

Now that we have our customers segmented into 5 different categories, we can gain further insight into customer behavior by using predictive models in conjuction with out RFM model.

## Data Pre-Processing and Exploring

In [ ]:
df_rfm2 = df_rfm[["Recency", "Frequency", "Monetary"]]
df_rfm2.head()

In [ ]:
df_rfm2.describe()

### i. Feature Correlations

Heatmap

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df_rfm2.corr(), annot=True)

### ii. Visualize Feature Distributions

In [ ]:
sns.pairplot(df_rfm2, aspect=2, height=4)

### iii. Data Normalization

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(df_rfm2)
rfm_scaled = pd.DataFrame(rfm_scaled, columns=["Recency", "Frequency", "Monetary"])
rfm_scaled.describe()

2. Plot normalized data with scatter matrix or pairplot. Also evaluate results.

In [ ]:
sns.pairplot(rfm_scaled, aspect=2, height=4)

## K-Means Implementation

We will try different cluster numbers and check their [silhouette coefficient](http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html). The silhouette coefficient for a data point measures how similar it is to its assigned cluster from -1 (dissimilar) to 1 (similar). 

### i. Defining the Optimal Number of Clusters

[The Elbow Method](https://en.wikipedia.org/wiki/Elbow_method_(clustering) 

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,10), timings=False)
visualizer.fit(rfm_scaled)
visualizer;

From the visualizer, optimal number of clusters is 5 and we can understand this from the various silhouette scores.

[Silhouette Coefficient](http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html)

In [ ]:
from sklearn.metrics import silhouette_score

for i in range(2,10):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(rfm_scaled)
    print(f"K Means Clustering --> n={i} and Silhouette Score =",silhouette_score(rfm_scaled, kmeans.labels_))

### ii. Model Fitting

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(rfm_scaled)

### iii. Visualize the Clusters

1. Creating a scatter plot and selecting cluster centers

In [ ]:
plt.figure(figsize=(15,8))
rfm_scaled["cluster"] = kmeans.labels_
centroids = kmeans.cluster_centers_
sns.scatterplot(rfm_scaled.iloc[:,0], rfm_scaled.iloc[:, 1], c=rfm_scaled["cluster"], cmap="rainbow")
sns.scatterplot(list(centroids[:,0]), list(centroids[:,1]), s=500)
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D # 3D Scatterplot

%matplotlib inline
fig = plt.figure(figsize=(20,10))
ax = plt.axes(projection='3d')
ax.scatter3D(rfm_scaled.iloc[:,0], rfm_scaled.iloc[:, 2], c=rfm_scaled["cluster"], cmap="rainbow", s=50, alpha=0.5)
ax.scatter3D(list(centroids[:,0]), list(centroids[:,1]), s=500, color="black")
ax.view_init(30, 35)
plt.show()

### KMeans Clustering with Logaritm Normalization

In [ ]:
rfm_scaled = np.log1p(df_rfm2)
rfm_scaled.fillna(0, inplace=True)

In [ ]:
sns.pairplot(rfm_scaled, aspect=2, height=4)

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,10), timings=False)
visualizer.fit(rfm_scaled)
visualizer;

In [ ]:
for i in range(2,10):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(rfm_scaled)
    print(f"K Means Clustering --> n={i} and Silhouette Score =",silhouette_score(rfm_scaled, kmeans.labels_))

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(rfm_scaled)

In [ ]:
plt.figure(figsize=(15,8))
rfm_scaled["cluster"] = kmeans.labels_
centroids = kmeans.cluster_centers_
sns.scatterplot(rfm_scaled.iloc[:,0], rfm_scaled.iloc[:, 1], c=rfm_scaled["cluster"], cmap="rainbow")
sns.scatterplot(list(centroids[:,0]), list(centroids[:,1]), s=500)
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
fig = plt.figure(figsize=(20,10))
ax = plt.axes(projection='3d')
ax.scatter3D(rfm_scaled.iloc[:,0], rfm_scaled.iloc[:, 2], c=rfm_scaled["cluster"], cmap="rainbow", s=50, alpha=0.3)
ax.scatter3D(list(centroids[:,0]), list(centroids[:,2]), s=750, color="black", alpha=1)
ax.view_init(50, 35)
plt.show()

 2. Visualizing Cluster Id vs Recency, Cluster Id vs Frequency and Cluster Id vs Monetary using Box plot. 

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(25,8))

sns.boxplot(x="cluster", y="Recency", data=rfm_scaled, ax=ax[0])
sns.boxplot(x="cluster", y="Frequency", data=rfm_scaled, ax=ax[1])
sns.boxplot(x="cluster", y="Monetary", data=rfm_scaled, ax=ax[2])

We can determine clusters from the above plots. Lower Recency and higher Frequency and higher Monetary gives as perfect customer and the reverse tells us that we have to look that customers deeply.

### iv. Assigning the Label

In [ ]:
def assign_label(x):
  if x == 0:
    return "Best"
  elif x == 2:
    return "Loyal"
  elif x == 4:
    return "Promising"
  elif x == 1:
    return "Needs Attention"
  else:
    return "Requires Action"

In [ ]:
rfm_scaled["cluster_label"] = rfm_scaled["cluster"].apply(assign_label)

**Conclusion**

- Cluster 0 : The first cluster belongs to the "Best Customers" segment which we saw earlier as they purchase recently (R=4), frequent buyers (F=4), and spent the most (M=4)

- Cluster 2 : These customers are also have low recency, high frequency and monetary purchase. They are not best customers but we can say that they are loyal. 

- Cluster 4 : These customers doesn'come recently but they were coming before and company can lose them. Company should pay more attention these customers and why they don't come recently. 

- Cluster 1 :  clusters can be interpreted as passer customers as their last purchase is long ago (R<=1),purchased very few (F>=2 & F < 4) and spent little (M>=4 & M < 4).Company has to come up with new strategies to make them permanent members. Low value customers

- Cluster 0 : The last cluster is more related to the "Almost Lost" segment as they Haven’t purchased for some time(R=1), but used to purchase frequently and spent a lot.

### v. Conclusion

Discuss your final results. Compare your own labels from the Customer Segmentation with the labels found by K-Means.

In [ ]:
print("Number of customers that rfm label and cluster label's equal:", (df_rfm["rfm_level"] == rfm_scaled["cluster_label"]).sum())
print("Number of customers that rfm label and cluster label's are not equal:", (df_rfm["rfm_level"] != rfm_scaled["cluster_label"]).sum())

plt.title("RFM Label & Cluster Label", c="blue", size=14)
sns.countplot((df_rfm["rfm_level"] == rfm_scaled["cluster_label"]));

In [ ]:
df_compare = pd.concat([df_rfm["rfm_level"], rfm_scaled["cluster_label"]], axis=1)

plt.figure(figsize=(20,12))
plt.title("Countplot of Cluster Label by RMF Level", c="blue", size=14)
sns.countplot(x="cluster_label", hue="rfm_level", data=df_compare)

In [ ]:
df_compare = pd.concat([df_rfm["rfm_level"], rfm_scaled["cluster_label"]], axis=1)

plt.figure(figsize=(20,12))
plt.title("Countplot of RMF Level by Cluster Label", c="blue", size=14)
sns.countplot(x="rfm_level", hue="cluster_label", data=df_compare)

From the above plot we can conclude that both method determine "Best" customers. However Kmeans gives less value for "Requires Action" cluster and predict most of the rfm_level's loyal label as promising label. 

# 5. Create Cohort & Conduct Cohort Analysis
[Cohort Analysis](https://medium.com/swlh/cohort-analysis-using-python-and-pandas-d2a60f4d0a4d) is specifically useful in analyzing user growth patterns for products. In terms of a product, a cohort can be a group of people with the same sign-up date, the same usage starts month/date, or the same traffic source.
Cohort analysis is an analytics method by which these groups can be tracked over time for finding key insights. This analysis can further be used to do customer segmentation and track metrics like retention, churn, and lifetime value.

For e-commerce organizations, cohort analysis is a unique opportunity to find out which clients are the most valuable to their business. by performing Cohort analysis you can get the following answers to the following questions:

- How much effective was a marketing campaign held in a particular time period?
- Did the strategy employ to improve the conversion rates of Customers worked?
- Should I focus more on retention rather than acquiring new customers?
- Are my customer nurturing strategies effective?
- Which marketing channels bring me the best results?
- Is there a seasonality pattern in Customer behavior?
- Along with various performance measures/metrics for your organization.

## Future Engineering

### i. Extract the Month of the Purchase
First we will create a function, which takes any date and returns the formatted date with day value as 1st of the same month and Year.

In [ ]:
def day_first(x):

  return dt.datetime(x.year, x.month, 1)

Now we will use the function created above to convert all the invoice dates into respective month date format.

In [ ]:
df["InvoiceMonth"] = df["InvoiceDate"].apply(day_first)
df["InvoiceMonth"].head()

In [ ]:
df["CohortMonth"] = df.groupby("CustomerID")["InvoiceMonth"].transform(min)
df["CohortMonth"].head()

### ii. Calculating time offset in Months i.e. Cohort Index:
Calculating time offset for each transaction will allows us to report the metrics for each cohort in a comparable fashion.
First, we will create 4 variables that capture the integer value of years, months for Invoice and Cohort Date using the get_date_int() function which we'll create it below.

In [ ]:
def get_date_int(x):
  
  y = x.dt.year
  m = x.dt.month
  d = x.dt.day
  return y, m, d

We will use this function to extract the integer values for Invoice as well as Cohort Date in 3 seperate series for each of the two columns

In [ ]:
 inv_year, inv_month, inv_day = get_date_int(df["InvoiceMonth"])
 coh_year, coh_month, coh_day = get_date_int(df["CohortMonth"])

Use the variables created above to calculate the difference in days and store them in cohort Index column.

In [ ]:
years_diff = inv_year - coh_year
months_diff = inv_month - coh_month

df['CohortIndex'] = years_diff * 12 + months_diff + 1 # Find retention for monthly 
df["CohortIndex"].describe()

## 1st Cohort: User number & Retention Rate

### i. Pivot Cohort and Cohort Retention

In [ ]:
cohort_data = df.groupby(["CohortMonth", "CohortIndex"])["CustomerID"].nunique().reset_index() # Unique number of customers in cohort month and cohort index.
pivot_cohort = pd.pivot_table(data=cohort_data, index="CohortMonth", columns="CohortIndex", values="CustomerID") # creating pivot table
retention_rate = (pivot_cohort.divide(pivot_cohort.iloc[:,0], axis=0)).round(2) # retention rate
retention_rate.index = retention_rate.index.strftime('%Y-%m')

### ii. Visualize analysis of cohort 1 using seaborn and matplotlib modules

In [ ]:
plt.figure(figsize=(18,10))
sns.heatmap(data=retention_rate, cmap="Blues", annot=True, fmt=".0%", vmax=0.8, vmin=0.0)
plt.yticks(rotation=0);

## 2nd Cohort: Average Quantity Sold

### i. Pivot Cohort and Cohort Retention

In [ ]:
cohort_data = df.groupby(["CohortMonth", "CohortIndex"])["Quantity"].mean().reset_index() # Quantity mean for cohort month and cohort index
avg_quantity = pd.pivot_table(data=cohort_data, index="CohortMonth", columns="CohortIndex", values="Quantity") 
avg_quantity.index = avg_quantity.index.strftime('%Y-%m')

### ii. Visualize analysis of cohort 2 using seaborn and matplotlib modules

In [ ]:
plt.figure(figsize=(18,10))
sns.heatmap(data=avg_quantity, cmap="Reds", annot=True)
plt.yticks(rotation=0);

## 3rd Cohort: Average Sales


### i. Pivot Cohort and Cohort Retention

In [ ]:
cohort_data = df.groupby(["CohortMonth", "CohortIndex"])["Cost"].mean().reset_index() # Cost mean for cohort month and cohort index
avg_sales = pd.pivot_table(data=cohort_data, index="CohortMonth", columns="CohortIndex", values="Cost")
avg_sales.index = avg_sales.index.strftime('%m-%Y')

### ii. Visualize analysis of cohort 3 using seaborn and matplotlib modules

In [ ]:
plt.figure(figsize=(18,10))
sns.heatmap(data=avg_sales, cmap="Greens", annot=True)
plt.yticks(rotation=0);